# PRÁCTICA 1: Extracción y búsqueda de información textual


In [1]:
import scrapy
import json
import random
from bs4 import BeautifulSoup
from dateutil import parser

# Parte 1- Compilar datos de documentos web

En este apartado nos piden extraer documentos utilizando RSS, JSON-LD y HTML puro. Cada uno de estos formatos tiene características y utilidades diferentes:

**RSS (Really Simple Syndication)** es un formato XML estandarizado diseñado específicamente para la sindicación de contenidos, es decir, para que sitios web publiquen automáticamente sus actualizaciones de forma estructurada. Los feeds RSS contienen metadatos organizados en etiquetas predefinidas como `<title>`, `<link>`, `<description>`, `<pubDate>`, etc., lo que facilita enormemente la extracción automática de noticias, artículos de blogs y otros contenidos que se actualizan frecuentemente. La mayoría de medios de comunicación y blogs ofrecen feeds RSS en URLs predecibles (como `/rss`, `/feed`, o `/rss.xml`), lo que los convierte en la fuente ideal para scrapers de noticias al proporcionar datos ya estructurados y estandarizados.

**JSON-LD (JSON for Linking Data)** es un formato de datos estructurados que se incrusta dentro del HTML de una página web, típicamente dentro de etiquetas `<script type="application/ld+json">`. Es parte del vocabulario Schema.org y se utiliza para proporcionar metadatos semánticos sobre el contenido de la página de forma que tanto humanos como máquinas (especialmente motores de búsqueda) puedan entenderlo mejor. En el contexto de artículos de noticias, JSON-LD suele contener información estructurada como `headline`, `datePublished`, `author`, `articleBody`, `image`, etc., siguiendo el esquema `NewsArticle` o `Article`. Extraer datos desde JSON-LD es muy eficiente porque ya están en formato JSON (fácil de parsear) y suelen ser más completos y precisos que los metadatos HTML tradicionales.

Para **RSS y JSON-LD** la extracción es relativamente sencilla siguiendo los ejemplos de prácticas de clase y adaptándolos a nuestras páginas, ya que ambos formatos están estandarizados y estructurados. El verdadero desafío está en el **scraping de HTML puro**, que requiere mayor personalización al ser cada página web diferente: debemos inspeccionar manualmente la estructura del DOM, identificar los selectores CSS o XPath correctos para cada campo (título, contenido, fecha, autor), manejar diferentes layouts, y adaptar el código específicamente para cada sitio objetivo.

Para implementar estos scrapers utilizaremos **Scrapy**, un framework de Python diseñado específicamente para web scraping y crawling a gran escala. Scrapy funciona mediante la definición de **Spiders** (arañas), que son clases Python que heredan de `scrapy.Spider` y definen cómo navegar por un sitio web y cómo extraer datos de sus páginas. Cada Spider tiene un método `parse()` que recibe un objeto `response` (la respuesta HTTP de una página) y debe devolver o bien datos extraídos (como diccionarios Python o items de Scrapy) o bien nuevas peticiones (objetos `Request`) para seguir crawleando otras URLs. La extracción de datos se realiza mediante **selectores CSS o XPath** aplicados sobre el HTML: por ejemplo, `response.css('h1.title::text').get()` extrae el texto del primer `<h1>` con clase `title`. Scrapy maneja automáticamente aspectos complejos como el seguimiento de enlaces.

Se han elegido documentos relacionados con la **economía**

## RSS

In [52]:
def to_yyyy_mm_dd(date_str: str) -> str:
    return parser.parse(date_str).date().isoformat()

In [ ]:
class RSSSpider (scrapy.Spider):

    # Es obligatorio poner el nombre del Spider
    name = 'RSS'

    # Estas son las URLs donde empieza a buscar el Spider
    start_urls = ['https://feeds.elpais.com/mrss-s/pages/ep/site/elpais.com/section/economia/portada',
                  'https://www.abc.es/rss/2.0/economia/',
                  'https://www.eleconomista.es/rss/rss-seleccion-ee.php',
                  'https://nadaesgratis.es/feed',
                  'https://www.lavanguardia.com/rss/economia.xml']

    # para evitar que el sitio te bloquee por usar scrapy es interesante cambiar el USER_AGENT
    # El user agent por defecto de Scrapy cuando hace una petición es
    # Scrapy/VERSION (+https://scrapy.org)
    custom_settings = {
        'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    }


    def parse (self, response):
        """
        @inherit

        @param self
        @param response
        """

        #Buscamos todos los elementos en el archivo XML con la etiqueta <item>
        for item in response.css ('item'):
            #Obtenemos por cada elemento <item> el texto del subelemento <link>
            url = str (item.css ('link::text').get ()).strip()
            #Obtenemos por cada elemento <item> el texto del subelemento <title>. Además co el BeautifulSoup
            #procesamos el texto en html y nos quedamos con el texto
            title = BeautifulSoup(str(item.css ('title::text').get()), 'html.parser').get_text().strip()
            #Obtenemos por cada elemento <item> el texto del subelemento <title>
            content = BeautifulSoup(str(item.css ('description::text').get()), 'html.parser').get_text().strip()
            # Fecha
            o_date = (str(item.css ('pubDate::text').get())).strip()
            date = to_yyyy_mm_dd(o_date)


            #Imprimimos la información obtenida para comprobar lo que estamos extrayendo
            print ("-------------------------")
        
            print ('URL:' + url)
            print ('Título:' + title)
            print ('Descripción:' + content)
            print (f'Fecha: {o_date} -> {date}')

            if content is None or content == '':
                print ("La descripción está vacía")
                continue
                
            print ("-------------------------")

            data = {
                'date': date,
                'url' : url,
                'title': title,
                'content': content,
                'document_type': 'rss'
            }

            #Creamos para cada item un fichero json y para ello obtenemos un número aleatorio.
            filename = str(random.random()).replace(".","") + url[11:18].replace(".","") + ".json"

            # Si tenemos descripción, url y título entonces lo guardamos a disco en la carpeta 'rss'
            if content and title and url:
                print("Guardando en disco el fichero: " + filename)
                with open ('rss/' + filename, 'w', encoding='utf-8') as f:
                    json.dump (data, f, ensure_ascii=False, indent = 4)

In [ ]:
import os
import scrapy
from scrapy.crawler import CrawlerProcess

# Creamos un proceso de Crawler podemos poner distintas settings que están definidas en la documentación.
# Entre ellas podemos ocular los logs del proceso de Crawling.
process = CrawlerProcess(settings={
    "LOG_ENABLED": False,
    # Used for pipeline 1
})

# Como se ha definido anteriormente en el RSSCrawler, los ficheros se van a almacenar en la carpeta "rss"
# Comprobamos que existe la carpeta y si no existe la creamos
if (not os.path.exists('rss')):
    os.mkdir('rss')

# Creamos el proceso con el RSSSpider
process.crawl(RSSSpider)
# Ejecutamos el Crawler
process.start()

RuntimeError: This event loop is already running

-------------------------
URL:https://www.eleconomista.es/actualidad/noticias/13712482/01/26/castilla-y-leon-prorroga-las-medidas-cautelares-contra-la-dermatosis-nodular-contagiosa-hasta-el-31-de-enero.html
Título:Castilla y León prorroga las medidas cautelares contra la Dermatosis Nodular Contagiosa hasta el 31 de enero
Descripción:La Junta de Castilla y León ha prorrogado las medidas cautelares contra la Dermatosis Nodular Contagiosa en la Comunidad hasta el próximo 31 de enero. Las medidas afectarán únicamente, tal y como hasta el día de hoy, a las ferias y mercados de ganado bovino en las que los animales procedan de más de una única explotación.
Fecha: Fri, 02 Jan 2026 18:08:50 +0100 -> 2026-01-02
-------------------------
Guardando en disco el fichero: 036296553232729534elecon.json
-------------------------
URL:https://www.eleconomista.es/mercados-cotizaciones/noticias/13712463/01/26/el-salario-de-christine-lagarde-en-el-bce-en-el-punto-de-mira-cobra-un-558-mas-que-su-sueldo-ofic

: 

## JSON-LD


In [ ]:
class LaRazonEconomiaSpider(scrapy.Spider):
    name = 'larazon'
    allowed_domains = ['larazon.es']
    start_urls = ['https://www.larazon.es/economia/']

    custom_settings = {
        'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    }

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.count = 0   # contador de ficheros procesados

    def parse(self, response):
        # Detener si se ha alcanzado el límite de 300 documentos
        if self.count >= 300:
            return

        url = response.url.strip()

        # Extraer artículos JSON-LD
        for item in response.css('script[type="application/ld+json"]'):
            if self.count >= 300:
                return

            try:
                data = json.loads(item.css('::text').get())
            except (json.JSONDecodeError, TypeError):
                continue

            if "articleBody" in data:
                title = BeautifulSoup(str(data['headline']), 'html.parser').get_text().strip()
                content = BeautifulSoup(str(data['articleBody']), 'html.parser').get_text().strip()
                o_date = str(data.get('datePublished', ''))
                date = to_yyyy_mm_dd(o_date)

                if content and title:
                    print("-------------------------")
                    print(url)
                    print(title)
                    print(date)
                    print(f"Documentos guardados: {self.count + 1}")
                    print("-------------------------")

                    filename = str(random.random()).replace(".", "") + ".json"
                    data_out = {
                        'url': url,
                        'title': title,
                        'content': content,
                        'date': date,
                        'document_type': 'json-ld'
                    }

                    with open('json-ld/' + filename, 'w', encoding='utf-8') as f:
                        json.dump(data_out, f, ensure_ascii=False, indent=4)
                        self.count += 1  # aumentamos el contador global

                    if self.count >= 300:
                        return

        # Seguir enlaces (solo si no hemos llegado al límite)
        if self.count < 300:
            for next_page in response.css('a'):
                href = str(next_page.css('::attr(href)').get())
                if href and '/economia/' in href:
                    yield response.follow(next_page, self.parse)



In [ ]:
import os
import scrapy
from scrapy.crawler import CrawlerProcess

# Creamos un proceso de Crawler podemos poner distintas settings que están definidas en la documentación.
# Entre ellas podemos ocular los logs del proceso de Crawling.
process = CrawlerProcess(settings={
    "LOG_ENABLED": False,
    # Used for pipeline 1
})

# Como se ha definido anteriormente en el RSSCrawler, los ficheros se van a almacenar en la carpeta "json-ld"
# Comprobamos que existe la carpeta y si no existe la creamos
if (not os.path.exists('json-ld')):
    os.mkdir('json-ld')

# Creamos el proceso con el RSSSpider
process.crawl(LaRazonEconomiaSpider)
# Ejecutamos el Crawler
process.start()

RuntimeError: This event loop is already running

-------------------------
https://www.larazon.es/economia/fernando-santiago-ano-despues-notificaciones-electronicas-siguen-ignorando-vida-personas_202601026957fea5af09df50109976cc.html
Fernando Santiago: “Un año después, las notificaciones electrónicas siguen ignorando la vida de las personas”
2026-01-02
Documentos guardados: 1
-------------------------
-------------------------
https://www.larazon.es/economia/flota-pesquera-rusa-prepara-invadir-caladero-marroqui_2026010169566fe6ea66eb735323ac36.html
La flota pesquera rusa se prepara para "invadir" el caladero marroquí
2026-01-01
Documentos guardados: 2
-------------------------
-------------------------
https://www.larazon.es/economia/ibex-35-dispara-nuevos-maximos_202512306954044f22f0db7daf016c82.html
El Ibex 35 se dispara a nuevos máximos
2025-12-30
Documentos guardados: 3
-------------------------
-------------------------
https://www.larazon.es/economia/gonzalo-bernardos-economista-explica-como-tener-jubilacion-tranquila-mejor-pla

: 

## HTML

Este spider de Scrapy extrae artículos de economía del sitio web de Radio Televisión Canaria (RTVC). El proceso se divide en dos fases: 

Primero, el método `parse()` navega por la página principal de economía (`https://rtvc.es/cat/rtvc-es/economia/`) extrayendo los enlaces a artículos individuales mediante selectores CSS que buscan elementos `<a>` con atributo `rel="bookmark"` (que identifica enlaces permanentes a artículos) excluyendo aquellos con clase `td-image-wrap` (que son las imágenes miniatura). Para evitar duplicados, se ignora el primer enlace encontrado ya que en esta web específica suele ser redundante. Una vez extraídos los enlaces de la página actual, el spider implementa **paginación automática**: busca el elemento `<span class="current">` que contiene el número de página actual, lo incrementa en 1, y busca el enlace `<a class="page" title="N">` correspondiente a la siguiente página usando XPath, siguiendo ese enlace recursivamente hasta alcanzar la página 80 (límite establecido para obtener aproximadamente 800 artículos, dado que cada página contiene unos 10 artículos). 

En la segunda fase, el método `parse_article()` procesa cada artículo individual extrayendo: el título desde `<h1 class="tdb-title-text">`, la fecha desde el atributo `datetime` de la etiqueta `<time>` (que se normaliza al formato YYYY-MM-DD usando `dateutil.parser`), y el contenido completo del artículo navegando por el contenedor principal `<div>` que contiene las clases `td_block_wrap`, `tdb_single_content` y `td-post-content`. Para extraer el texto se iteran todos los párrafos `<p>` dentro de este contenedor, concatenando el texto de cada uno (incluyendo descendientes) y filtrando aquellos que sean demasiado cortos (menos de 10 caracteres) o que contengan etiquetas `<style>` o `<script>` que no son contenido real. Finalmente, cada artículo válido (con título, contenido y fecha) se guarda en un archivo JSON individual con nombre único generado a partir de la fecha y un número aleatorio para evitar colisiones.

In [ ]:
import scrapy
import json
import random
from bs4 import BeautifulSoup
from dateutil import parser

def to_yyyy_mm_dd(date_str: str) -> str:
    return parser.parse(date_str).date().isoformat()

class TVCanariaSpider(scrapy.Spider):
    name = 'tvcanaria'
    allowed_domains = ['rtvc.es']
    start_urls = ['https://rtvc.es/cat/rtvc-es/economia/']

    custom_settings = {
        'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    }

    def parse(self, response):
      """
      @inherit

      @param self
      @param response
      """

      # Obtenemos la URL de la web que estamos procesando
      url = str(response.request.url).strip()

      ignore_first = True
      for link in response.css ('a'):
        # if link tiene una etiqueta rel con valor "bookmark" y no tiene la clase "td-image-wrap"
        rel_attr = link.attrib.get('rel', '')      # puede no existir
        class_attr = link.attrib.get('class', '')  # puede no existir


        if 'bookmark' in rel_attr and 'td-image-wrap' not in class_attr:
            if ignore_first:
                ignore_first = False
                continue
            href = link.attrib.get("href")
            if href:
                yield response.follow(href, callback=self.parse_article)

      # span con clase current representa el número de página actual
      current_span = response.css('span.current::text').get()
      if current_span:
          try:
            current_page = int(current_span.strip())
            print(f'Página actual: {current_page}')
            if current_page == 80: # En cada página hay 10 artículos, así que 80 páginas son 800 artículos
                return
            next_page_title = str(current_page + 1)

            # Buscar el enlace <a class="page" title="next_page_title">
            next_link = response.xpath(
                f'//a[@class="page" and @title="{next_page_title}"]/@href'
            ).get()

            if next_link:
                yield response.follow(next_link, callback=self.parse)

          except ValueError:
            # Si el texto no es un número, ignoramos la paginación
            pass

        


    def parse_article(self, response):
        url = response.url

        # Título (en RTVC suele estar en h1) :contentReference[oaicite:1]{index=1}
        title = response.css("h1.tdb-title-text::text").get()
        if title:
            title = title.strip()
        o_date = response.css("time::attr(datetime)").get()
        date = to_yyyy_mm_dd(o_date)
 
        # Seleccionar el contenedor principal (solo con clases estables)
        content_div = response.xpath(
            '//div[contains(@class, "td_block_wrap") and '
            'contains(@class, "tdb_single_content") and '
            'contains(@class, "td-post-content")]'
        )

        paragraph_texts = []

        # Iterar sobre cada <p> dentro del contenedor
        for p in content_div.xpath('.//p'):
            # Obtener el texto completo de este <p>, incluyendo descendientes
            texts = p.xpath('.//text()').getall()
            full_text = ''.join(texts).strip()

            # Filtrar: solo considerar si hay texto real, más de 10 caracteres
            if len(full_text) > 10:
                # Opcional: verificar que el <p> no contenga <style> o <script>
                has_bad_tags = p.xpath('.//style | .//script').get()
                if not has_bad_tags:
                    paragraph_texts.append(full_text)

        full_content = '\n'.join(paragraph_texts)

        if title and full_content and date:
            data = {
                'date': date,
                'url' : url,
                'title': title,
                'content': full_content
            }
            with open(f'tvcanaria/{date}_{str(random.random()).replace(".", "")}.json', 'w', encoding='utf-8') as f:
                json.dump(data, f, ensure_ascii=False, indent=4)

In [ ]:
import scrapy, os
from scrapy.crawler import CrawlerProcess

# Creamos un proceso de Crawler podemos poner distintas settings que están definidas en la documentación.
# Entre ellas podemos ocular los logs del proceso de Crawling.
process = CrawlerProcess(settings={
    "LOG_ENABLED": False,
    # Used for pipeline 1
})

# Como se ha definido anteriormente en el RSSCrawler, los ficheros se van a almacenar en la carpeta "rss"
# Comprobamos que existe la carpeta y si no existe la creamos
if (os.path.exists('tvcanaria')== False):
    os.mkdir('tvcanaria')

# Creamos el proceso con el RSSSpider
process.crawl(TVCanariaSpider)
# Ejecutamos el Crawler
process.start()

RuntimeError: This event loop is already running

Página actual: 1
Página actual: 2
Página actual: 3
Página actual: 4
Página actual: 5
Página actual: 6
Página actual: 7
Página actual: 8
Página actual: 9
Página actual: 10
Página actual: 11
Página actual: 12
Página actual: 13
Página actual: 14
Página actual: 15
Página actual: 16
Página actual: 17
Página actual: 18
Página actual: 19
Página actual: 20
Página actual: 21
Página actual: 22
Página actual: 23
Página actual: 24
Página actual: 25
Página actual: 26
Página actual: 27
Página actual: 28
Página actual: 29
Página actual: 30
Página actual: 31
Página actual: 32
Página actual: 33
Página actual: 34
Página actual: 35
Página actual: 36
Página actual: 37
Página actual: 38
Página actual: 39
Página actual: 40
Página actual: 41
Página actual: 42
Página actual: 43
Página actual: 44
Página actual: 45
Página actual: 46
Página actual: 47
Página actual: 48
Página actual: 49
Página actual: 50
Página actual: 51
Página actual: 52
Página actual: 53
Página actual: 54
Página actual: 55
Página actual: 56
P

: 

# Parte 2 - Buscador tradicional

Conceptos básicos de elasticsearch:

ElasticSearch es una base de datos distribuida donde los datos (documentos) se envían/gestionan como JSON a través de la API, orientada a búsqueda (texto completo) más que a transacciones. La API es una REST API y esta diseñada para que las peticiones sean con JSON.

- Índice (index): parecido a una “tabla” (ej: noticias).
- Documento (document): parecido a una “fila” (una noticia en JSON).
- Mapping: el “esquema” del índice, dice qué tipo tiene cada campo (texto, fecha, keyword, etc.).
- Analyzer: cómo se procesa el texto al indexar y buscar en un índice (tokenización, minúsculas, stemming…).

Es parecido a mongodb con las colecciones y documentos, en este caso una colección es el índice, dentro del índice guardamos los diferentes documentos.

Una vez que tienes los documentos en un índice, puedes hacer queries para extraer los datos. Las queries se hacen con [Query DSL](https://www.elastic.co/docs/explore-analyze/query-filter/languages/querydsl) un lenguaje que usa JSON para buscar en la base de datos. Es en este JSON de la query donde especificamos los parámetros de la búsqueda (términos, operadores, filtros por fecha, orden, paginación, resaltado, sugerencias, etc.).

En este apartado de la práctica nos están pidiendo que programemos un buscador sobre los documentos que hemos recopilado que cumpla ciertas caracteristicas (operadores lógicos, rangos de fechas...). Por lo tanto, lo que tenemos que hacer es:

1. Crear el índice para nuestros JSON de noticias
2. Cargar los JSON al índice
3. Programar el "buscador", que va a ser una función a la cual le pasamos los términos que queremos buscar y demás configuraciones que nos piden y nos devuelve los documentos que encanjan con la query. Básicamente una función que nos construya el JSON de la query



### 1º) Crear el índice donde vamos a guardar los datos

In [1]:
from elasticsearch import Elasticsearch

ES_URL = "http://localhost:9200"
INDEX_NAME = "noticias" 

client = Elasticsearch(ES_URL)

# Comprobamos la conexión
print(client.info())

{'name': '3c6a729cbcef', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'e3B9hV5wRwulVqQHW38dJA', 'version': {'number': '8.12.1', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '6185ba65d27469afabc9bc951cded6c17c21e3f3', 'build_date': '2024-02-01T13:07:13.727175297Z', 'build_snapshot': False, 'lucene_version': '9.9.2', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


Para crear el índice definimos:

* **Settings de análisis**: cómo se va a procesar el texto cuando indexamos y cuando buscamos.
Un analyzer se compone de uan serie de elementos opcionales: un tokenizer (que divide el texto en tokens iniciales) y una serie de filters, que modifican, añaden o eliminan esos tokens. Los filters son transformaciones secuenciales que se aplican a los tokens ya generados.
* **Mapping**: qué tipo tiene cada campo (fecha, texto, keyword…), lo que afecta a búsquedas, filtros, ordenaciones, etc.

### Analyzer en español (stemming + stopwords + normalización)

Creamos un analyzer personalizado (`es_analyzer`) compuesto por:

* `lowercase`: convierte todo a minúsculas (evita que “Gobierno” y “gobierno” sean distintos).
* `asciifolding`: normaliza acentos/diacríticos (ej. “educación” ≈ “educacion”).
* `spanish_stop`: elimina **stopwords** del español (como “de”, “la”, “y”…), reduciendo ruido.
* `spanish_stemmer` (`light_spanish`): aplica **stemming**, reduciendo palabras a su raíz aproximada (ej. “economía”, “económico”, “económicos” tienden a coincidir mejor).

Esto cubre el requisito de la práctica de **uso de stemming** y mejora la recuperación de documentos cuando el usuario escribe variaciones morfológicas.

### Mapping de campos (qué guardamos y cómo)

Definimos la estructura de las noticias que recopilamos antes:

Conviene entender un poco los [tipos de datos disponibles](https://www.elastic.co/docs/reference/elasticsearch/mapping-reference/field-data-types) en elasticsearch antes de crear el índice. Si investigamos un poco, veremos estos tres tipos, que son los que nos hacen falta para nuestros JSONs:

>keyword, which is used for structured content such as IDs, email addresses, hostnames, status codes, zip codes, or tags.

>text, the traditional field type for full-text content such as the body of an email or the description of a product

>date, JSON doesn’t have a date data type, so dates in Elasticsearch can either be:
strings containing formatted dates, e.g. "2015-01-01" or "2015/01/01 12:10:30".
a number representing milliseconds-since-the-epoch.
a number representing seconds-since-the-epoch

El tipo `keyword` almacena cadenas sin procesar, ideales para búsquedas exactas, filtrado o agregaciones. Text se utiliza para texto completo y se somete a análisis (como tokenización, stemming o eliminación de stopwords). Finalmente, el tipo date permite almacenar fechas en formatos estándar y facilita operaciones como filtrado por rangos temporales.

* `url` como `keyword`: campo exacto, útil para identificar de forma única y evitar análisis, en la misma documentación recomiendan poner campos como este (url) en keyword.
* `title` como `text` con `es_analyzer`: permite búsqueda por texto completo con stemming/stopwords.
  * Además añadimos `title.raw` como `keyword`: una versión “sin analizar” del título. Esto es útil si luego queremos **ordenar alfabéticamente por título**, porque ordenar por `text` no es correcto/eficiente.
* `content` como `text` con `es_analyzer`: el cuerpo principal de la noticia, preparado para búsqueda full-text.
* `date` como `date`: imprescindible para **filtrar por rangos de fechas** (otro requisito de la práctica). Acepta formatos ISO y `yyyy-MM-dd`.
* `document_type` como `keyword`: etiqueta o categoría (ej. “news”, “blog”, “press”), perfecta para filtros exactos.



In [2]:
# Si existe, lo borramos para empezar limpio
if client.indices.exists(index=INDEX_NAME):
    client.indices.delete(index=INDEX_NAME)

# Crear índice con analyzer en español (stemming + stopwords + lowercase + asciifolding)
index_body = {
    "settings": {
        "analysis": {
            "filter": {
                "spanish_stop": {"type": "stop", "stopwords": "_spanish_"},
                "spanish_stemmer": {"type": "stemmer", "language": "light_spanish"}
            },
            "analyzer": {
                "es_analyzer": {
                    "type": "custom",
                    "tokenizer": "standard",
                    "filter": [
                        "lowercase",
                        "asciifolding",
                        "spanish_stop",
                        "spanish_stemmer"
                    ]
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "url": {"type": "keyword"},
            "title": {
                "type": "text",
                "analyzer": "es_analyzer",
                "fields": {
                    "raw": {"type": "keyword"}  # Para luego ordenar por título
                }
            },
            "content": {"type": "text", "analyzer": "es_analyzer"},
            "date": {
                "type": "date",
                "format": "strict_date_optional_time||yyyy-MM-dd"
            },
            "document_type": {"type": "keyword"}
        }
    }
}

client.indices.create(index=INDEX_NAME, body=index_body)
print(f"Índice creado: {INDEX_NAME}")

Índice creado: noticias


### 2º) Cargar los JSON dentro del índice

Una vez creado el índice con su *mapping* y *analyzer*, el siguiente paso es **meter dentro de Elasticsearch (en el índice) todos los documentos** (nuestras noticias en formato JSON). La idea de esta celda es:

1. Recorrer las carpetas donde tenemos los ficheros (`json-ld/`, `rss/`, `tvcanaria/`)
2. Leer cada `.json`
3. Preparar una carga masiva (*bulk*) para indexar rápido y de forma eficiente

---

#### Lectura de ficheros y preparación de documentos

* `glob()` se usa para listar todos los archivos `*.json` dentro de cada carpeta.
* Cada fichero se abre y se carga como diccionario Python con `json.load()`.

Además, vamos contando:

* `total_files`: cuántos ficheros hemos encontrado en total
* `actions`: lista con todas las operaciones que vamos a enviar a Elasticsearch en bloque

---

#### Evitar duplicados con un `_id` estable

En Elasticsearch, si no especificas `_id`, se genera uno aleatorio y **podrías indexar duplicados** sin darte cuenta.

Por eso se define `make_id(doc)`, crea la clave en base a la `url`:

Esto hace que si el mismo documento se intenta indexar dos veces, tendrá el mismo `_id` y Elasticsearch lo sobrescribe en vez de duplicarlo.

---

#### Indexación masiva con `helpers.bulk`

En lugar de hacer un `client.index(...)` por cada documento (muy lento), se usa:

* `helpers.bulk(client, actions)`

Esto envía **muchos documentos en una sola operación** o en lotes internos, lo que es mucho más eficiente.

Después:

* `client.indices.refresh(...)` fuerza a que los documentos recién indexados estén disponibles para búsqueda inmediatamente

Finalmente:

* `client.count(...)` se usa para comprobar cuántos documentos hay indexados y confirmar que la carga ha ido bien.


In [3]:
import os
import json
import hashlib
from glob import glob
from elasticsearch import helpers

BASE_DIR = "." 
FOLDERS = ["json-ld", "rss", "tvcanaria"]

def make_id(doc: dict) -> str:
    """
    Genera un _id estable para evitar duplicados usando la URL
    """
    key = doc.get("url")
    return hashlib.md5(key.encode("utf-8")).hexdigest()

actions = []
total_files = 0

for folder in FOLDERS:
    pattern = os.path.join(BASE_DIR, folder, "*.json")
    files = glob(pattern)
    total_files += len(files)

    for fp in files:
        with open(fp, "r", encoding="utf-8") as f:
            doc = json.load(f)

        # Acción bulk
        actions.append({
            "_index": INDEX_NAME,
            "_id": make_id(doc),
            "_source": doc
        })

print(f"Ficheros encontrados: {total_files}")
print(f"Documentos preparados para indexar: {len(actions)}")

# Bulk insert
helpers.bulk(client, actions)
client.indices.refresh(index=INDEX_NAME)

count = client.count(index=INDEX_NAME)["count"]
print(f"Indexación completada. Documentos en el índice '{INDEX_NAME}': {count}")

Ficheros encontrados: 1282
Documentos preparados para indexar: 1282
Indexación completada. Documentos en el índice 'noticias': 1282


### 3º) Crear función "buscador"

En este apartado ya construimos el buscador como tal: una función que recibe lo que escribiría el usuario (términos, operadores, fechas, paginación, etc.) y genera una query en Query DSL (JSON) para enviarla a Elasticsearch.

Para realizar búsquedas en Elasticsearch desde Python utilizamos `client.search()`, que acepta varios parámetros clave: `index` especifica el índice donde buscar, `query` contiene la consulta en formato Query DSL (un diccionario Python que se serializa a JSON), `size` limita el número de resultados devueltos, `from_` permite la paginación, `sort` define el orden de los resultados, y `highlight` resalta los términos encontrados en los campos especificados. 

La respuesta es un diccionario con la estructura estándar de Elasticsearch: `hits.total.value` indica cuántos documentos coinciden, y `hits.hits` contiene el array de resultados, donde cada elemento incluye `_score` (un número decimal que representa la relevancia del documento según el algoritmo BM25, siendo mayor score más relevante, este score se calcula automáticamente basándose en la frecuencia de los términos, la rareza de las palabras en el índice y la longitud del campo), `_source` (documento original con todos sus campos), y opcionalmente `highlight` (fragmentos de texto donde aparecen los términos buscados, rodeados por etiquetas HTML personalizables como `<mark>` para facilitar su visualización, se puede configurar el tamaño de los fragmentos, el número máximo de fragmentos por campo, y si queremos el campo completo o solo extractos relevantes). Las queries DSL permitem combinar múltiples condiciones mediante operadores booleanos (`bool`), búsquedas de texto completo (`match`, `match_phrase`), filtros exactos (`term`, `range`) y agregaciones.

Vamos a ver algunos ejemplos de consultas sencillas para entenderlo bien

In [6]:
# Búsqueda “manual” con bool (must/should/must_not)
manual_bool_query = {
    "bool": { # bool combina múltiples condiciones de búsqueda
        "must": [ # "must" = obligatorio (y afecta al score)
            {"match": {"content": {"query": "banca", "operator": "AND"}}} # busca "banca" en content, todas las palabras deben aparecer
        ],
        "should": [ # "should" = opcional, pero sube score si coincide
            {"match": {"title": {"query": "BBVA"}}},
            {"match": {"title": {"query": "Sabadell"}}}
        ],
        "must_not": [ # "must_not" = excluye resultados
            {"match_phrase": {"content": "Banco Santander"}} # excluye si aparece la frase exacta "Banco Santander"
        ],
        "minimum_should_match": 1  # obliga a que al menos uno de los should se cumpla
    }
}

resp_manual = client.search(
    index=INDEX_NAME,
    query=manual_bool_query,
    size=5,
    sort=[{"date": {"order": "desc"}}],
    highlight={
        "pre_tags": ["<mark>"],
        "post_tags": ["</mark>"],
        "fields": {
            "title": {"number_of_fragments": 0},
            "content": {"fragment_size": 160, "number_of_fragments": 2}
        }
    }
)

print(f"Resultados encontrados (búsqueda manual): {resp_manual['hits']['total']['value']}")
for hit in resp_manual["hits"]["hits"]:
    print(f"ID: {hit['_id']}, Score: {hit['_score']}")
    print(f"Title: {hit['highlight'].get('title', [hit['_source']['title']])[0]}")
    for fragment in hit['highlight'].get('content', []):
        print(f"...{fragment}...")
    print("-----")

Resultados encontrados (búsqueda manual): 6
ID: c98b99e0a2af9f1fe97fb8c92455555e, Score: None
Title: La banca vuelve a la casilla de salida de las fusiones en el año del fracaso de la opa al <mark>Sabadell</mark>
...El BBVA necesita tener más peso en Europa y el <mark>banco</mark> vallesano, un plan B tras salir debilitado de la contienda...
-----
ID: c6e131ed9ad377b46bd979fe8acaff27, Score: None
Title: Fracasa la OPA hostil del <mark>BBVA</mark> al <mark>Sabadell</mark>
...El consejero del <mark>Banco</mark> Sabadell en Galicia, Juan Manuel Vieites, ha considerado este viernes que el fracaso de la OPA que el BBVA planeaba hacer sobre el Sabadell...
...El fracaso de la OPA del BBVA sobre el <mark>Banco</mark> Sabadell ha hecho descarrilar la fusión de dos entidades que hubieran dado lugar al segundo <mark>banco</mark> por negocio en España...
-----
ID: c223f14ddb0b77c04819bc3368d6a46d, Score: None
Title: El consejo del <mark>Sabadell</mark> rechaza por unanimidad la OPA de <mark>BBVA</

Mientras que las queries booleanas con `bool` requieren construir manualmente la estructura JSON con cláusulas `must`, `should` y `must_not`, el tipo `query_string` permite escribir la búsqueda de forma más natural y similar a un buscador web como Google. Con `query_string` el usuario puede escribir directamente una cadena de texto utilizando operadores: `AND` para requerir múltiples términos, `OR` para alternativas, `NOT` para exclusiones, paréntesis para agrupar condiciones, y comillas dobles para búsquedas de frases exactas (por ejemplo: `'banca AND (BBVA OR Sabadell) NOT "Banco Santander"'`). El parámetro `fields` permite especificar en qué campos buscar, pudiendo aplicar boost (como `title^2` que multiplica por 2 el score de coincidencias en el título, dándole más relevancia). `default_operator` establece el operador por defecto entre términos cuando no se especifica explícitamente (AND o OR), `lenient: True` hace que la query sea más tolerante a errores de sintaxis o tipos de datos incompatibles, y `analyze_wildcard: True` permite el uso de comodines como `*` o `?` en la búsqueda.

In [7]:
# query_string: AND/OR/NOT + paréntesis + comillas

google_like_query = {
    "query_string": {
        "query": 'banca AND (BBVA OR Sabadell) NOT "Banco Santander"', # consulta
        "fields": ["title^2", "content"], # title tiene más peso (boost 2x)
        "default_operator": "AND", # por defecto todas las palabras deben aparecer
        "lenient": True, # ignora errores de parsing
        "analyze_wildcard": True # permite comodines en la consulta
    }
}

resp_qs = client.search(
    index=INDEX_NAME,
    query=google_like_query,
    size=5,
    sort=[{"date": {"order": "desc"}}],
    highlight={
        "pre_tags": ["<mark>"],
        "post_tags": ["</mark>"],
        "fields": {
            "title": {"number_of_fragments": 0},
            "content": {"fragment_size": 160, "number_of_fragments": 2}
        }
    }
)

print(f"Resultados encontrados (query_string): {resp_qs['hits']['total']['value']}")
for hit in resp_qs["hits"]["hits"]:
    print(f"ID: {hit['_id']}, Score: {hit['_score']}")
    print(f"Title: {hit['highlight'].get('title', [hit['_source']['title']])[0]}")
    for fragment in hit['highlight'].get('content', []):
        print(f"...{fragment}...")
    print("-----")

Resultados encontrados (query_string): 13
ID: 7ca72d23ea78adf14d80d6bf85f8a1e6, Score: None
Title: El Ibex registra un año récord desde 1993 y se coloca como la mejor Bolsa de Occidente
...Durante gran parte del ejercicio, la <mark>banca</mark> ha actuado como el auténtico motor del mercado», afirma.Así las cosas, el desempeño de los <mark>bancos</mark> determina en gran medida...
...Por sectores, no solo la <mark>banca</mark> ha tenido un ejercicio boyante en Bolsa....
-----
ID: c98b99e0a2af9f1fe97fb8c92455555e, Score: None
Title: La <mark>banca</mark> vuelve a la casilla de salida de las fusiones en el año del fracaso de la opa al <mark>Sabadell</mark>
...El <mark>BBVA</mark> necesita tener más peso en Europa y el <mark>banco</mark> vallesano, un plan B tras salir debilitado de la contienda...
-----
ID: c6e131ed9ad377b46bd979fe8acaff27, Score: None
Title: Fracasa la OPA hostil del <mark>BBVA</mark> al <mark>Sabadell</mark>
...El consejero del <mark>Banco</mark> <mark>Sabadell</mark> 

Este ejemplo integra todas las técnicas anteriores en una query completa. Combina una búsqueda de texto con `query_string` dentro de la cláusula `must` de un operador `bool`, junto con un filtro temporal en la cláusula `filter`. 

La diferencia clave entre `must` y `filter` es que `must` afecta al score de relevancia (por lo que se usa para condiciones de búsqueda), mientras que `filter` solo filtra resultados sin afectar al score (ideal para rangos de fechas, categorías o cualquier condición binaria sí/no, además de ser más rápido al cacheable). 

El filtro `range` con `gte` (greater than or equal) y `lte` (less than or equal) delimita las fechas entre las que queremos buscar. La paginación se implementa con `from_` (posición inicial) y `size` (cantidad de resultados), calculando el offset como `(página - 1) × tamaño`. 

El parámetro `sort` ordena los resultados por fecha descendente (más recientes primero), sobrescribiendo el orden natural por `_score`. Finalmente, `highlight` resalta los términos encontrados tanto en título (campo completo con `number_of_fragments: 0`) como en contenido (mostrando hasta 3 fragmentos de 160 caracteres cada uno).

In [8]:
# Filtro por fechas en filter + sort + highlight + paginación

date_from = "2025-12-15"
date_to   = "2026-01-02"

page = 2
size = 5
from_ = (page - 1) * size

query_with_dates = {
    "bool": {
        "must": [
            {
                "query_string": {
                    "query": "banca OR finanzas",
                    "fields": ["title^2", "content"],
                    "default_operator": "AND",
                    "lenient": True
                }
            }
        ],
        "filter": [
            {"range": {"date": {"gte": date_from, "lte": date_to}}}
        ]
    }
}

resp_dates = client.search(
    index=INDEX_NAME,
    query=query_with_dates,
    from_=from_,
    size=size,
    sort=[{"date": {"order": "desc"}}],
    highlight={
        "pre_tags": ["<mark>"],
        "post_tags": ["</mark>"],
        "fields": {
            "title": {"number_of_fragments": 0},
            "content": {"fragment_size": 160, "number_of_fragments": 3}
        }
    }
)

print(f"Página {page} (size={size})")
print(f"Resultados encontrados (filtro por fechas): {resp_dates['hits']['total']['value']}")
for hit in resp_dates["hits"]["hits"]:  
    print(f"ID: {hit['_id']}, Score: {hit['_score']}")
    print(f"Title: {hit['highlight'].get('title', [hit['_source']['title']])[0]}")
    for fragment in hit['highlight'].get('content', []):
        print(f"...{fragment}...")
    print("-----")

Página 2 (size=5)
Resultados encontrados (filtro por fechas): 38
ID: 9d46edf66f80609c11a29034766c7121, Score: None
Title: El Gobierno aspira a que un español acceda por primera vez a la Presidencia del BCE
...Este 2026 arranca la carrera para renovar los cuatro principales puestos de la cúpula del <mark>Banco</mark> Central Europeo, y el Gobierno jugará fuerte en dicha competición...
-----
ID: f893f5198313490b3356198c7f45c2c8, Score: None
Title: El salario de Christine Lagarde en el BCE, en el punto de mira: cobra un 55,8% más que su sueldo oficial
...La presidenta del <mark>Banco</mark> Central Europeo (BCE), Christine Lagarde, cobra más de un 50% de su salario oficial, según un análisis que ha publicado en exclusiva el diario...
...Según el medio, el salario real que percibe la presidenta del <mark>banco</mark> central es de 726.000 euros anuales, un 55,8% más que su salario básico oficial, el que comunica...
-----
ID: 7e379641a3f7dd868fc6e25a0cf41555, Score: None
Title: El País Vasc

Ahora que entendemos como funcionan las queries, podemos ver como resolver los requerimientos que nos piden para el buscador:

* **Operadores lógicos AND/OR/NOT**: se soportan gracias a `query_string`, que permite escribir consultas tipo Google con sintaxis lógica.
* **Rango de fechas**: se implementa con un `range` dentro de `bool.filter`, que filtra sin afectar al score.
* **Ordenación**: mediante `sort_field` y `sort_dir`.
* **Highlight**: resalta coincidencias en `title` y `content` usando `<mark>...</mark>`.
* **Paginación**: con `from_ = (page-1)*size` y `size`.
* **Stemming**: no se programa aquí; viene del `analyzer` del índice (el `es_analyzer` creado antes).
* **Fuzzy**: si `fuzzy=True`, añade un `multi_match` con `fuzziness: AUTO` en un `should` para tolerar errores.
* **Term suggester**: si `enable_suggest=True`, añade un `suggest` para proponer correcciones ortográficas.

---

Cómo se construye la Query DSL internamente

1. **Validaciones básicas**: evita queries vacías, normaliza page/size y el `sort_dir`.
2. **Cláusula principal `query_string`**:

   * Busca sobre `title` y `content` (con boost `title^2` para dar más peso a coincidencias en título).
   * `default_operator: AND`: si el usuario escribe “banca bbva”, se interpreta como `banca AND bbva`.
3. **`bool` con `must` y `filter`**:

   * `must`: contiene el `query_string` (lo que determina el score).
   * `filter`: contiene el rango de fechas (filtrado puro, sin score).
4. **Fuzzy opcional**:

   * Se mete en `should` (no obligatorio), para recuperar más resultados si hay typos sin romper la búsqueda principal.
5. **Highlight opcional**:

   * `title`: sin fragmentos (devuelve el título completo resaltado).
   * `content`: devuelve hasta 3 fragmentos de ~160 caracteres.
6. **Sort**:

   * Si se ordena por `_score`, se usa score.
   * Si se ordena por campo, se especifica `unmapped_type` para evitar errores si algún doc no tiene ese campo.
   * Nota: si ordenas por texto, lo correcto es usar `title.raw` (keyword), no `title` (text).
7. **Suggest opcional**:

   * Limpia la query de símbolos (paréntesis, comillas, operadores) y se queda con tokens.
   * Lanza term suggester sobre `title` y `content`.
8. **Ejecución**:

   * `client.search(...)` con `query`, `from_`, `size`, `sort`, `highlight` y `suggest`.
9. **Normalización de salida**:

   * Convierte la respuesta en un diccionario más cómodo: lista de results, total, pages, took, suggestions…
   * Además incluye `raw_query` para depurar / enseñar el JSON que se envía.

---

In [9]:
import re
from typing import Any

def search_news(
    user_query: str,
    date_from: str | None = None,
    date_to: str | None = None,
    page: int = 1,
    size: int = 10,
    sort_field: str = "date",
    sort_dir: str = "desc",
    highlight: bool = True,
    fuzzy: bool = False,
    enable_suggest: bool = True
) -> dict[str, Any]:
    """
    Buscador sobre el índice INDEX_NAME usando el cliente Elasticsearch `client`.

    Requisitos cubiertos:
      1) AND/OR/NOT -> user_query vía query_string
      2) Rango fechas -> range filter en campo `date`
      3) Orden -> sort_field / sort_dir
      4) Highlight -> highlight en title/content
      5) Paginación -> from/size calculado por page/size
      6) Stemming -> lo hace el analyzer del índice (es_analyzer) al indexar/buscar
      7) Fuzzy -> si fuzzy=True añade una cláusula fuzzy (multi_match fuzziness AUTO)
      8) Suggester -> term suggester si enable_suggest=True
    """

    # Validaciones básicas
    if not user_query or not user_query.strip():
        raise ValueError("user_query no puede estar vacío")

    page = max(1, int(page))
    size = max(1, int(size))
    sort_dir = (sort_dir or "desc").lower()
    if sort_dir not in ("asc", "desc"):
        sort_dir = "desc"

    from_ = (page - 1) * size

    # Query principal con query_string (AND/OR/NOT dentro del string)
    # Campos: title con boost, content normal
    query_string_clause = {
        "query_string": {
            "query": user_query,
            "fields": ["title^2", "content"],
            "default_operator": "AND",      # si el usuario no pone operadores, el espacio actúa como AND
            "lenient": True,
            "analyze_wildcard": True
        }
    }

    bool_query: dict[str, Any] = {
        "bool": {
            "must": [query_string_clause],
            "filter": [],
        }
    }

    # Rango de fechas (filter, no afecta al score)
    if date_from or date_to:
        r: dict[str, Any] = {}
        if date_from:
            r["gte"] = date_from
        if date_to:
            r["lte"] = date_to
        bool_query["bool"]["filter"].append({"range": {"date": r}})

    # Fuzzy
    # query_string soporta fuzziness usando ~ (ej: "sabadell~1"),
    # pero como aquí queremos un "switch" fuzzy=True, añadimos un should fuzzy
    # para tolerar typos sin obligar al usuario a poner ~ manualmente.
    if fuzzy:
        bool_query["bool"].setdefault("should", [])
        bool_query["bool"]["should"].append({
            "multi_match": {
                "query": user_query,
                "fields": ["title^2", "content"],
                "fuzziness": "AUTO",
                "operator": "AND"
            }
        })
        # No ponemos minimum_should_match porque ya hay must; el should solo mejora recall/score.

    # Highlight
    highlight_obj = None
    if highlight:
        highlight_obj = {
            "pre_tags": ["<mark>"],
            "post_tags": ["</mark>"],
            "fields": {
                "title": {"number_of_fragments": 0},
                "content": {"fragment_size": 160, "number_of_fragments": 3}
            }
        }

    # Orden (sort)
    # Si ordenas por texto, usa "title.raw" (keyword) y no "title" (text).
    if sort_field in ("score", "_score"):
        sort_obj = ["_score"] if sort_dir == "desc" else [{"_score": {"order": "asc"}}]
        sort_obj = [{"_score": {"order": sort_dir}}]
    else:
        unmapped_type = "date" if sort_field == "date" else "keyword"
        sort_obj = [{sort_field: {"order": sort_dir, "unmapped_type": unmapped_type}}]

    # Term suggester (opcional)
    suggest_obj = None
    if enable_suggest:
        # Para suggest conviene quitar operadores y símbolos y quedarse con tokens "normales"
        tokens = re.findall(r"[0-9A-Za-zÁÉÍÓÚÜÑáéíóúüñ]+", user_query)
        suggest_text = " ".join(tokens).strip()

        if suggest_text:
            suggest_obj = {
                "title_suggest": {
                    "text": suggest_text,
                    "term": {
                        "field": "title",
                        "suggest_mode": "popular",
                        "min_word_length": 3
                    }
                },
                "content_suggest": {
                    "text": suggest_text,
                    "term": {
                        "field": "content",
                        "suggest_mode": "popular",
                        "min_word_length": 3
                    }
                }
            }

    # Ejecutar búsqueda
    # Requiere que existan `client` e `INDEX_NAME` en tu notebook (de las celdas anteriores).
    resp = client.search(
        index=INDEX_NAME,
        query=bool_query,
        from_=from_,
        size=size,
        sort=sort_obj,
        highlight=highlight_obj,
        suggest=suggest_obj
    )

    # Normalizar salida
    hits = resp.get("hits", {}).get("hits", [])
    total = resp.get("hits", {}).get("total", {}).get("value", 0)

    results = []
    for h in hits:
        src = h.get("_source", {}) or {}
        results.append({
            "score": h.get("_score"),
            "url": src.get("url"),
            "title": src.get("title"),
            "date": src.get("date"),
            "content": src.get("content"),
            "document_type": src.get("document_type"),
            "source_folder": src.get("source_folder"),
            "highlight": h.get("highlight", {})
        })

    # Sugerencias en formato fácil
    suggestions = {}
    if "suggest" in resp and resp["suggest"]:
        for key, arr in resp["suggest"].items():
            # arr suele ser una lista con 1 entrada
            opts = []
            for entry in arr:
                for opt in entry.get("options", []):
                    # term suggester devuelve "text" como sugerencia
                    if "text" in opt:
                        opts.append(opt["text"])
            # quitar duplicados manteniendo orden
            seen = set()
            uniq = []
            for o in opts:
                if o not in seen:
                    seen.add(o)
                    uniq.append(o)
            suggestions[key] = uniq

    return {
        "page": page,
        "size": size,
        "total": total,
        "pages": (total + size - 1) // size,
        "took_ms": resp.get("took"),
        "results": results,
        "suggestions": suggestions,
        "raw_query": { 
            "query": bool_query,
            "sort": sort_obj,
            "highlight": highlight_obj,
            "suggest": suggest_obj,
            "from": from_,
            "size": size
        }
    }


In [10]:
res = search_news(
    user_query='banca AND (BBVA OR Sabadell) NOT "Banco Santander"',
    date_from="2025-12-15",
    date_to="2026-01-02",
    page=1,
    size=5,
    sort_field="date",
    sort_dir="desc",
    highlight=True,
    fuzzy=False,
    enable_suggest=True
)

print(f"Total resultados: {res['total']}")
for r in res["results"]:
    print(f"Title: {r['highlight'].get('title', [r['title']])[0]}")
    print(f"Date: {r['date']}")
    for fragment in r['highlight'].get('content', []):
        print(f"...{fragment}...")
    print("-----")



Total resultados: 2
Title: El Ibex registra un año récord desde 1993 y se coloca como la mejor Bolsa de Occidente
Date: 2026-01-02
..., con crecimientos de valoración incluso superiores al cien por cien en algunos casos como el Santander, <mark>BBVA</mark> o Caixabank ....
...Durante gran parte del ejercicio, la <mark>banca</mark> ha actuado como el auténtico motor del mercado», afirma.Así las cosas, el desempeño de los <mark>bancos</mark> determina en gran medida...
...Por sectores, no solo la <mark>banca</mark> ha tenido un ejercicio boyante en Bolsa....
-----
Title: La <mark>banca</mark> vuelve a la casilla de salida de las fusiones en el año del fracaso de la opa al <mark>Sabadell</mark>
Date: 2026-01-01
...El <mark>BBVA</mark> necesita tener más peso en Europa y el <mark>banco</mark> vallesano, un plan B tras salir debilitado de la contienda...
-----


In [5]:
from elasticsearch import Elasticsearch, helpers
import json

# Nombre del índice a exportar
index_name = INDEX_NAME

# Abrimos un archivo para guardar los datos
with open('indice_ej2.json', 'w') as file:
    # Usamos el helper scan para obtener todos los documentos del índice
    for doc in helpers.scan(client, index=index_name):
        # Escribimos cada documento en el archivo en formato JSON
        file.write(json.dumps(doc) + '\n')

print("Exportación completada.")


Exportación completada.


# Parte 3 – Indexación con embeddings vectoriales (búsqueda semántica, k-NN Search)

En la Parte 2 montamos un buscador **léxico**: Elasticsearch decide qué documentos son relevantes en función de coincidencias de palabras (y su análisis: stemming, stopwords, etc.). El problema típico de este enfoque es que:

* Si el usuario escribe sinónimos o reformula la idea, puede que **no haya coincidencia literal**.
* Dos textos pueden hablar de lo mismo con vocabulario distinto, y el buscador tradicional los puede “perder”.

Para ampliar el buscador, en esta parte añadimos **búsqueda semántica (k-NN search)** mediante **embeddings vectoriales**: representaciones numéricas del texto (vectores) que capturan significado. Con esto, podemos recuperar documentos “parecidos” conceptualmente aunque no compartan exactamente las mismas palabras mediante el algoritmo k-NN que viene implementado en ElasticSearch.

### 1) Generación de embeddings con un modelo preentrenado

Cargamos un modelo de `sentence-transformers`:

```python
model = SentenceTransformer('hiiamsid/sentence_similarity_spanish_es')
```

Este tipo de modelos transforma un texto (por ejemplo, un título o un contenido) en un vector de dimensión fija. Es decir:

* Entrada: `"El Gobierno aprueba nuevas medidas económicas"`
* Salida: `[-0.12, 0.34, ..., 0.08]` (vector numérico)

La dimensión del vector depende del modelo, y por eso la calculamos dinámicamente con:

```python
DIMS = model.get_sentence_embedding_dimension()
```

Esto es importante porque Elasticsearch necesita saber **cuántos números tendrá el vector** para definir el campo `dense_vector`.

In [6]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('hiiamsid/sentence_similarity_spanish_es')

DIMS = model.get_sentence_embedding_dimension()
print("Dims:", DIMS)

/home/jose/master/tgine/P3_Scrapy/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dims: 768


### 2) Modificar el índice para almacenar vectores (`dense_vector`)

Para poder guardar embeddings dentro de los documentos, Elasticsearch necesita un campo especial de tipo:

* `dense_vector`: un array de floats de tamaño fijo (dims)

En esta práctica añadimos dos campos vectoriales:

* `title_embedding`: embedding del título
* `content_embedding`: embedding del contenido completo (o su resumen/recorte si fuera muy largo)

Esto permite comparar similitud semántica a distintos niveles: a veces el título ya basta, y otras el contenido aporta más contexto.

En Elasticsearch, para hacer **búsqueda k-NN eficiente** (aproximada) no basta con guardar el vector: hay que indicarle que lo indexe como estructura vectorial.

Por eso en el mapping se usa:

* `"index": True`: activa indexación vectorial para búsqueda k-NN.
* `"similarity": "cosine"`: la métrica de similitud será coseno (muy típica en embeddings).
* `"index_options": {"type": "hnsw", ...}`: usa el algoritmo **HNSW** (Hierarchical Navigable Small World), un estándar práctico para búsqueda aproximada de vecinos más cercanos.

Parámetros principales:

* `m`: controla cuántas conexiones crea el grafo (más alto = más memoria/precisión).
* `ef_construction`: controla el esfuerzo al construir el índice (más alto = más lento al indexar, pero mejor recall).

En práctica/entorno académico suelen ponerse valores como `m=16` y `ef_construction=100` porque dan buen equilibrio.

La función `put_mapping` nos permite añadir campos a un índice ya creado.

In [7]:
# Añadimos un nuevo campo al mapping
# Para búsqueda vectorial aproximada (kNN) conviene index:true + HNSW.
put_mapping_body = {
    "properties": {
        "title_embedding": {
            "type": "dense_vector",
            "dims": DIMS,
            "index": True,
            "similarity": "cosine",
            "index_options": {
                "type": "hnsw",
                "m": 16,
                "ef_construction": 100
            }
        },
        "content_embedding": {
            "type": "dense_vector",
            "dims": DIMS,
            "index": True,
            "similarity": "cosine",
            "index_options": {
                "type": "hnsw",
                "m": 16,
                "ef_construction": 100
            }
        }
    }
}

client.indices.put_mapping(index=INDEX_NAME, body=put_mapping_body)
print(f"Mapping actualizado en '{INDEX_NAME}' con 'title_embedding' y 'content_embedding' ({DIMS} dims).")

Mapping actualizado en 'noticias' con 'title_embedding' y 'content_embedding' (768 dims).


### 3) Generar embeddings y actualizar los documentos del índice (bulk update)

Una vez que el índice ya tiene definidos los campos `dense_vector` (`title_embedding` y `content_embedding`), el siguiente paso es **rellenar esos campos** para cada documento ya indexado. Esta celda hace exactamente eso:

1. Recorre todos los documentos existentes en Elasticsearch.
2. Extrae `title` y `content`.
3. Genera embeddings con `sentence-transformers`.
4. Actualiza cada documento con los vectores, usando operaciones masivas (*bulk*) para que sea eficiente.

Para recorrer un índice completo, `helpers.scan()` es lo más práctico, en vez de `search`:

* Está pensado para **iterar muchos documentos**.
* Evita problemas de paginación manual.
* Consume menos memoria y es más estable para colecciones grandes.

En `iter_docs_title_content()` hacemos un “generador” que va devolviendo:

* `_id` del documento (clave en Elasticsearch)
* `title`
* `content`

Además, el `query` especifica `_source: ["title", "content"]` para traer solo los campos necesarios.

El embedding es lo más caro computacionalmente. Por eso no codificamos documento por documento, sino en bloques:

```python
BATCH_SIZE = 128
```

Se acumulan listas:

* `batch_ids`: ids de docs
* `batch_titles`: títulos
* `batch_contents`: contenidos

Cuando llegamos al tamaño del batch, procesamos.

En vez de llamar dos veces al modelo (una para títulos y otra para contenidos), hacemos una sola inferencia:

```python
texts = batch_titles + batch_contents
vecs = model.encode(texts, normalize_embeddings=True)
```

Luego se separan los resultados:

* `title_vecs`: primera mitad
* `content_vecs`: segunda mitad

Esto es más eficiente porque reduces overhead de llamadas y aprovechas mejor el procesamiento vectorizado.

`normalize_embeddings=True` normaliza los vectores (norma 1). Con esto:

* La similitud coseno se vuelve más estable y rápida de calcular (coseno = producto punto).
* Es una buena práctica cuando se compara con `"similarity": "cosine"` en Elasticsearch.

Como los documentos ya existen en el índice, no hacemos `index` de nuevo: hacemos **update**.

Cada operación que se mete en `updates` es:

* `_op_type: "update"`
* `_id`: el documento a modificar
* `"doc": {...}`: solo los campos nuevos a añadir/actualizar

Así, no reescribimos todo el documento, solo añadimos:

* `title_embedding`
* `content_embedding`

Luego se ejecuta:

```python
helpers.bulk(client, updates, request_timeout=180)
```

El `request_timeout` se aumenta porque:

* Bulk updates con vectores pueden tardar (tamaño de payload alto).
* Queremos evitar timeouts del cliente.

Si el número total de documentos no es múltiplo de `BATCH_SIZE`, quedará un batch “incompleto”.
Por eso al final se repite la misma lógica dentro de:

```python
if batch_ids:
    ...
```

---

Al terminar:

* `client.indices.refresh(...)` fuerza a que los cambios sean visibles inmediatamente.
* Se imprime el total de documentos actualizados (`count`), que debería coincidir con el número de documentos en el índice.

A partir de este punto, **cada noticia del índice ya tiene su representación semántica** en forma de embeddings, y ya podemos hacer:

* búsquedas k-NN (similitud)
* comparación con búsquedas léxicas
* búsqueda híbrida (léxica + semántica)


In [8]:
from elasticsearch import helpers

BATCH_SIZE = 128  # content suele ser más largo, 128 es un valor seguro

def iter_docs_title_content(es_client, index_name):
    for doc in helpers.scan(
        es_client,
        index=index_name,
        query={"_source": ["title", "content"], "query": {"match_all": {}}},
        preserve_order=False
    ):
        _id = doc["_id"]
        src = doc.get("_source", {}) or {}
        title = src.get("title") or ""
        content = src.get("content") or ""
        yield _id, title, content

updates = []
count = 0

batch_ids = []
batch_titles = []
batch_contents = []

for _id, title, content in iter_docs_title_content(client, INDEX_NAME):
    batch_ids.append(_id)
    batch_titles.append(title)
    batch_contents.append(content)

    if len(batch_ids) >= BATCH_SIZE:
        # Una sola llamada al modelo: codificamos títulos + contenidos y luego separamos
        texts = batch_titles + batch_contents
        vecs = model.encode(texts, show_progress_bar=False, normalize_embeddings=True)

        title_vecs = vecs[:len(batch_titles)]
        content_vecs = vecs[len(batch_titles):]

        for doc_id, tvec, cvec in zip(batch_ids, title_vecs, content_vecs):
            updates.append({
                "_op_type": "update",
                "_index": INDEX_NAME,
                "_id": doc_id,
                "doc": {
                    "title_embedding": tvec.tolist(),
                    "content_embedding": cvec.tolist()
                }
            })

        helpers.bulk(client, updates, request_timeout=180)
        count += len(updates)

        updates.clear()
        batch_ids.clear()
        batch_titles.clear()
        batch_contents.clear()
        print(f"Actualizados: {count}")

# Último batch
if batch_ids:
    texts = batch_titles + batch_contents
    vecs = model.encode(texts, show_progress_bar=False, normalize_embeddings=True)

    title_vecs = vecs[:len(batch_titles)]
    content_vecs = vecs[len(batch_titles):]

    for doc_id, tvec, cvec in zip(batch_ids, title_vecs, content_vecs):
        updates.append({
            "_op_type": "update",
            "_index": INDEX_NAME,
            "_id": doc_id,
            "doc": {
                "title_embedding": tvec.tolist(),
                "content_embedding": cvec.tolist()
            }
        })

    helpers.bulk(client, updates, request_timeout=180)
    count += len(updates)

client.indices.refresh(index=INDEX_NAME)
print(f"Embeddings guardados en 'title_embedding' y 'content_embedding'. Total docs actualizados: {count}")

/tmp/ipykernel_96094/1798845966.py:49: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  helpers.bulk(client, updates, request_timeout=180)


Actualizados: 128
Actualizados: 256
Actualizados: 384
Actualizados: 512
Actualizados: 640
Actualizados: 768
Actualizados: 896
Actualizados: 1024
Actualizados: 1152
Actualizados: 1280
Embeddings guardados en 'title_embedding' y 'content_embedding'. Total docs actualizados: 1282


/tmp/ipykernel_96094/1798845966.py:77: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  helpers.bulk(client, updates, request_timeout=180)


In [9]:
from elasticsearch import Elasticsearch, helpers
import json

# Nombre del índice a exportar
index_name = INDEX_NAME

# Abrimos un archivo para guardar los datos
with open('indice_ej3.json', 'w') as file:
    # Usamos el helper scan para obtener todos los documentos del índice
    for doc in helpers.scan(client, index=index_name):
        # Escribimos cada documento en el archivo en formato JSON
        file.write(json.dumps(doc) + '\n')

print("Exportación completada.")

Exportación completada.


#### 4) k-NN Search

Seleccionamos el primer documento en el índice para sacar su `title` y `title_embedding` para hacer la prueba de búsqueda con k-nn y ver que resultados nos salen

In [14]:
res = client.search(
    index=INDEX_NAME,
    body={
        "query": {
            "match_all": {}  # o cualquier otra query
        },
        "size": 1  # solo un resultado
    }
)

primer_documento = res['hits']['hits'][0]

print("Título:", primer_documento["_source"]["title"])
print("Contenido:", primer_documento["_source"]["content"])
title_embedding_primer_doc = primer_documento["_source"]["title_embedding"]
content_embedding_primer_doc = primer_documento["_source"]["content_embedding"]

Título: La Agencia Tributaria Canaria estrena web
Contenido: La Agencia Tributaria Canaria presenta un nuevo portal web más intuitivo y seguro. Este estreno coincide con su décimo aniversario en pleno proceso de transformación y mejora continua, aseguran en un comunicado.
El objetivo es mejorar la experiencia del contribuyente en su relación con la administración tributaria canaria al facilitar de forma segura la asistencia digital que se le ofrece evitando dilaciones y desplazamientos innecesarios para hacer sus trámites.
En la dirección https://www3.gobiernodecanarias.org/tributos/atc/el contribuyente encontrará toda la información tributaria necesaria a un solo click. Además, podrá acceder directamente a la sede electrónica desde la página principal para realizar cualquier trámites online.
Un portal en el que pedir cita, presentar los diferentes modelos tributarios de los impuestos, transmisiones patrimoniales, actos jurídicos documentados, sucesiones y donaciones, tabaco, AIEM, com

Elasticsearch admite dos métodos de búsqueda kNN:

- kNN aproximado: Podemos ejecutar esta búsqueda de tres formas, `k-nn option`, `knn query` o `knn retriever`. Es rápido, escalable y adecuado para la mayoría de cargas de trabajo en producción. Ofrece baja latencia y buena precisión.
- kNN exacto (fuerza bruta): Utiliza una consulta `script_score` con una función de vector. Garantiza resultados precisos, pero no escala bien en conjuntos de datos grandes, ya que evalúa cada documento coincidente. Sin embargo, si se combina con un filtro que reduzca drásticamente el número de documentos, puede ofrecer un buen rendimiento. Como nosotros no tenemos muchos documentos, podemos usar esta búsqueda

In [15]:
q = {
  "query": {
    "script_score": { # usamos script_score para búsqueda vectorial, kNN exacto
      "query" : {
        "match_all": {}
      },
      "script": { # el script que calcula el score
        "source": "cosineSimilarity(params.queryVector, 'title_embedding') + 1.0", # cosineSimilarity devuelve [-1,1], sumamos 1 para evitar negativos
        "params": {
          "queryVector": title_embedding_primer_doc # el vector de consulta
        }
      }
    }
  }
}

print("Título original: ", primer_documento["_source"]["title"])

response = client.search(index=INDEX_NAME, body=q)

print("\nResultados similares por título:\n")

for hit in response['hits']['hits']:
    title = hit['_source'].get('title', 'Título no disponible')
    print(title)

Título original:  La Agencia Tributaria Canaria estrena web

Resultados similares por título:

La Agencia Tributaria Canaria estrena web
La Agencia Tributaria pone en marcha ‘Censos Web’ para agilizar las altas de los emprendedores
La Agencia Tributaria ya ha devuelto a los contribuyentes de IRPF más de 13.000 millones al cierre de 2025
Un autónomo, indignado en redes por la subida del 21% del IVA en las comisiones del datáfono: “Gracias Haciend
Mogán propone al Gobierno de Canarias que implante la tasa turística
La Agencia Tributaria devuelve 272 millones de euros en Canarias
Nace ‘Smart Data Canarias’, una plataforma inteligente para el sector turístico
Hacienda activa el servicio telefónico «Le Llamamos» para la declaración de la Renta 2024
Hacienda presenta las cuentas de 2026 a los grupos parlamentarios
Canarias volverá a gravar los cigarrillos electrónicos y el vapeo en los presupuestos de 2025


#### 5) Comparando búsquedas

Vamos a seleccionar 3 noticias manualmente y vamos a intentar recuperarlas con palabras clave y k-NN, a ver las diferencias. Las noticias que vamos a buscar son:

```
{
    "date": "2025-12-26",
    "url": "https://www.lavanguardia.com/economia/20251226/11395977/emisiones-bonos-verdes-alcanzan-niveles-record.html",
    "title": "Las emisiones de bonos verdes alcanzan niveles récord en 2025",
    "content": "La demanda de electricidad por los centros de datos y la IA dispara las inversiones"
}
```

```
{
    "date": "2024-07-10",
    "url": "https://rtvc.es/cnmc-expediente-sancionador-contra-endesa/",
    "title": "La CNMC inicia un expediente sancionador contra empresas de Endesa por posibles prácticas anticompetitivas",
    "content": "La Comisión Nacional de los Mercados y la Competencia (CNMC) ha iniciado un expediente sancionador contra varias empresas del Grupo Endesa por posibles prácticas anticompetitivas, entre ellas, haber podido dar un trato «discriminatorio y preferente» a la resolución de solicitudes, reclamaciones e incidencias de sus propias filiales, en detrimento de terceras empresas competidoras..."
}
```

```
{
    "url": "https://www.larazon.es/economia/barrios-donde-vivienda-solo-esta-alcance-muy-pocos-piso-90-metros-cuesta-media-139-millones-mas-caro-ranking_20251225694cef10ea66eb73531e7cbd.html",
    "title": "Los barrios donde la vivienda solo está al alcance de muy pocos: un piso de 90 metros cuesta de media 1,39",
    "content": "En las calles más emblemáticas de Madrid, en las costas de Baleares y en las urbanizaciones más caras de Málaga el ladrillo es un lujo al alcance de muy pocos. Hasta 10.000 euros puede llegar a costar el metro cuadrado en el distrito más caro de España Y más de 15.000 en el barrio más caro. En concreto, en el distrito de Salamanca, el enclave más lujoso de Madrid, el metro cuadrado cuesta de media 9.950 euros, en máximos históricos, según un análisis de Idealista...,
    "date": "2025-12-25",
    "document_type": "json-ld"
}
```


Para la búsqueda k-nn vamos a hacerlo por title, ya que en la búsqueda clásica es el campo que más peso tiene, para que la comparación sea lo más justa posible

In [26]:
def knn_by_title(title: str, k: int = 5) -> list[dict[str, Any]]:
    """
    Dada una cadena de texto `title`, devuelve los k documentos más similares
    en el índice INDEX_NAME usando el campo `title_embedding`.
    """

    # 1) Obtener embedding del título de consulta
    query_vector = model.encode([title], show_progress_bar=False, normalize_embeddings=True)[0]
    # 2) Construir query script_score (kNN exacto)
    script_score_query = {
        "script_score": {
            "query": {"match_all": {}},
            "script": {
                "source": "cosineSimilarity(params.queryVector, 'title_embedding') + 1.0",
                "params": {"queryVector": query_vector}
            }
        }
    }

    # 3) Ejecutar búsqueda
    resp = client.search(
        index=INDEX_NAME,
        query=script_score_query,
        size=k
    )

    hits = resp.get("hits", {}).get("hits", [])

    results = []
    for h in hits:
        s = h.get("_source", {}) or {}
        results.append({
            "id": h.get("_id"),
            "score": h.get("_score"),
            "url": s.get("url"),
            "title": s.get("title"),
            "date": s.get("date"),
            "content": s.get("content"),
            "document_type": s.get("document_type"),
            "source_folder": s.get("source_folder"),
        })

    return results

1. Las emisiones de bonos verdes alcanzan niveles récord en 2025

Para esta primera consulta vamos a usar las palabras clave "bonos" "verdes". En este caso como ambos términos se encuentran en el título ambas búsquedas lo encuentran el primero

In [33]:
knn_results = knn_by_title("bonos verdes", 8)
print("Resultados kNN por título:\n")
for r in knn_results:
    print(r["score"], r["title"])


Resultados kNN por título:

1.4767451 Las emisiones de bonos verdes alcanzan niveles récord en 2025
1.4446704 Santa Cruz agota todos los Bonos Consumo de su quinta edición
1.4223857 El euríbor alcanza mínimos anuales, anticipando un alivio para los hipotecados
1.406195 La Gomera activa una nueva campaña de Bonos Consumo
1.4056796 ¿Ha cotizado en varios regímenes? Este es el que financiará su jubilación
1.4016261 PwC y Deloitte lideran los rankings de M&A en España
1.3995506 Cabildo y Fauca activan una nueva compra de los bonos consumo ‘Yo compro en La Gomera’
1.3992808 La Generalitat aporta otros 73,5 millones al programa Moves III


In [ ]:
res = search_news(
    user_query='bonos verdes',
    page=1,
    size=5,
    sort_field="date",
    sort_dir="desc",
    highlight=False,
    fuzzy=False,
    enable_suggest=False
)

print(f"Total resultados: {res['total']}")
for r in res["results"]:
    print(f"Title: {r['highlight'].get('title', [r['title']])[0]}")
    print(f"Date: {r['date']}")
    for fragment in r['highlight'].get('content', []):
        print(f"...{fragment}...")
    print("-----")

Total resultados: 2
Title: Las emisiones de bonos verdes alcanzan niveles récord en 2025
Date: 2025-12-26
-----
Title: La bolsa española se recupera tras la pausa anunciada por Trump
Date: 2025-04-10
-----


2. La CNMC inicia un expediente sancionador contra empresas de Endesa por posibles prácticas anticompetitivas

Para esta segunda query usaremos las palabras clave "CNMC" "sancion" "Endesa". En este caso la búsqueda knn encuentra la noticia sin problema, mientras que la búsqueda tradicional no encuentra ningún documento, ni siquiera con fuzzy activado. Para que lo encuentre tenemos que poner "sancionador" en vez de "sancionar"

In [34]:
knn_results = knn_by_title("CNMC sancion Endesa", 8)
print("Resultados kNN por título:\n")
for r in knn_results:
    print(r["score"], r["title"])

Resultados kNN por título:

1.6092273 La CNMC inicia un expediente sancionador contra empresas de Endesa por posibles prácticas anticompetitivas
1.5293138 Competencia autoriza la opa del BBVA al Sabadell y deja la operación en manos del Gobierno
1.509892 Fracasa la OPA hostil del BBVA al Sabadell
1.4938885 Repsol lanza las primeras estaciones de servicio para repostar diésel 100% renovable
1.4929819 Apollo, Cheyne y Tikehau pactan con los acreedores sacar a flote Torraspapel y recapitalizarla
1.4925725 De Repsol a Iberdrola: estos son los primeros dividendos de 2026
1.4905899 PwC y Deloitte lideran los rankings de M&A en España
1.475949 Toy Planet y Eurekakids pactan impulsar el juguete educativo en España


In [ ]:
res = search_news(
    user_query='CNMC sancion Endesa', 
    page=1,
    size=5,
    sort_field="date",
    sort_dir="desc",
    highlight=False,
    fuzzy=False,
    enable_suggest=False
)

print(f"Total resultados: {res['total']}")
for r in res["results"]:
    print(f"Title: {r['highlight'].get('title', [r['title']])[0]}")
    print(f"Date: {r['date']}")
    for fragment in r['highlight'].get('content', []):
        print(f"...{fragment}...")
    print("-----")

Total resultados: 0


Si ponemos sancion sin fuzzy no lo detecta, si no lo ponemos tampoco

In [39]:
res = search_news(
    user_query='CNMC sancion Endesa', 
    page=1,
    size=5,
    sort_field="date",
    sort_dir="desc",
    highlight=False,
    fuzzy=True,
    enable_suggest=True
)

print(f"Total resultados: {res['total']}")
for r in res["results"]:
    print(f"Title: {r['highlight'].get('title', [r['title']])[0]}")
    print(f"Date: {r['date']}")
    for fragment in r['highlight'].get('content', []):
        print(f"...{fragment}...")
    print("-----")

Total resultados: 0


In [43]:
res = search_news(
    user_query='CNMC sancionador Endesa', 
    page=1,
    size=5,
    sort_field="date",
    sort_dir="desc",
    highlight=False,
    fuzzy=True,
    enable_suggest=True
)

print(f"Total resultados: {res['total']}")
for r in res["results"]:
    print(f"Title: {r['highlight'].get('title', [r['title']])[0]}")
    print(f"Date: {r['date']}")
    for fragment in r['highlight'].get('content', []):
        print(f"...{fragment}...")
    print("-----")

Total resultados: 1
Title: La CNMC inicia un expediente sancionador contra empresas de Endesa por posibles prácticas anticompetitivas
Date: 2024-07-10
-----


3. Los barrios donde la vivienda solo está al alcance de muy pocos: un piso de 90 metros cuesta de media 1,39

Para la última consulta usaremos las palabras "aumento" "vivienda". Ambas palabras están fuertemente relacionadas con la noticia pero "aumento" no se encuentra ni en el título ni en el contenido de la misma. En este caso ninguna consigue encontrar la noticia exacta, aunque si encuentran muchas noticias relacionadas con el tema

In [44]:
knn_results = knn_by_title("aumento vivienda", 20)
print("Resultados kNN por título:\n")
for r in knn_results:
    print(r["score"], r["title"])

Resultados kNN por título:

1.6553092 Nueva convocatoria de ayudas para construir viviendas de alquiler social
1.5876472 ¿Cuánto aumenta el precio de mi vivienda tras una reforma?
1.568481 La catástrofe de la vivienda: sólo las élites pueden alquilar y colectivos que en teoría no son vulnerables
1.5641491 Los fondos de la RIC se podrán utilizar para la construcción de viviendas
1.5451136 La calle más cara de Canarias para comprar una vivienda
1.5374706 El Gobierno de Canarias convoca las ayudas para construir viviendas protegidas para el alquiler
1.5370212 La comunidad de propietarios deberá respaldar los nuevos pisos turísticos
1.5358797 El precio de la vivienda nueva crece en Canarias
1.5272532 Este es el tipo de vivienda que más se vende en España este 2024
1.5170763 Esta es la nueva deducción por reformar una vivienda para la Renta en 2022
1.5135996 Acceder a una vivienda, una tarea casi imposible
1.5133481 Aumenta el coste en un 50% a la hora de reformar una vivienda
1.5099134 Gon

In [49]:
res = search_news(
    user_query='aumento vivienda', 
    page=1,
    size=15,
    sort_field="date",
    sort_dir="desc",
    highlight=False,
    fuzzy=True,
    enable_suggest=True
)

print(f"Total resultados: {res['total']}")
for r in res["results"]:
    print(f"Title: {r['highlight'].get('title', [r['title']])[0]}")
    print(f"Date: {r['date']}")
    for fragment in r['highlight'].get('content', []):
        print(f"...{fragment}...")
    print("-----")

Total resultados: 132
Title: Récord histórico en turismo: España recibe a 91,5 millones de visitantes internacionales que gastaron más que
Date: 2026-01-02
-----
Title: Baleares, epicentro de la compra de vivienda por extranjeros: cuatro de cada diez operaciones ya son de no residentes
Date: 2026-01-02
-----
Title: La compraventa de viviendas en Canarias cayó un 1,8% en octubre
Date: 2025-12-30
-----
Title: Javier Linares, asesor financiero: "El ciudadano medio tiene 17.000 euros en el banco y es el mayor error que
Date: 2025-12-26
-----
Title: Daniel Lacalle, economista: "Los salarios han subido un 17% pero la inflación acumulada un 24%, es decir, han
Date: 2025-12-23
-----
Title: Reducir la brecha habitacional dispararía en PIB español un 10%
Date: 2025-12-17
-----
Title: Gonzalo Bernardos, economista: "Al invertir en viviendas de lujo te la juegas, las mejores son las de clase
Date: 2025-12-15
-----
Title: En noviembre el IPC subió en Canarias un 0,2%
Date: 2025-12-12
-----
Title: C

**Conclusiones**

1. **Cuando hay coincidencia léxica directa, ambos enfoques funcionan bien**, pero con matices.
   En la consulta sobre *“bonos verdes”*, al estar ambos términos explícitamente en el título, tanto la búsqueda tradicional como la semántica recuperan correctamente la noticia objetivo en primera posición. Sin embargo, la búsqueda k-NN devuelve además otros resultados conceptualmente cercanos (bonos consumo, campañas de bonos), mientras que la búsqueda por palabras clave es más restrictiva y devuelve menos resultados, algunos de ellos menos relacionados semánticamente.

2. **La búsqueda semántica es claramente superior ante variaciones morfológicas o conceptuales**.
   En la consulta sobre *CNMC, sanción y Endesa*, la búsqueda tradicional falla al no existir coincidencia exacta entre “sanción” y “sancionador”, incluso usando fuzzy search y sugerencias. En cambio, la búsqueda k-NN identifica correctamente la noticia relevante, demostrando su capacidad para capturar similitud semántica más allá de la forma exacta de las palabras. Esto evidencia una limitación importante de los enfoques puramente léxicos en escenarios reales de lenguaje natural.

3. **Ambos métodos tienen limitaciones cuando los términos de consulta no aparecen ni explícita ni implícitamente en el texto**.
   En la consulta *“aumento vivienda”*, ninguna de las dos búsquedas recupera la noticia concreta sobre barrios exclusivos y precios elevados, ya que el término “aumento” no aparece ni en el título ni en el contenido. Aun así, ambos enfoques devuelven noticias relacionadas con el mercado inmobiliario. La diferencia es que la búsqueda tradicional recupera un volumen muy elevado de resultados poco específicos, mientras que la semántica (aunque tampoco acierta) tiende a priorizar resultados conceptualmente más próximos al significado global de la consulta.

4. **En términos de precisión y robustez, la búsqueda semántica ofrece una experiencia más flexible**.
   La búsqueda k-NN se muestra más tolerante a sinónimos, variaciones lingüísticas y formulaciones incompletas, lo que la hace especialmente adecuada para consultas imprecisas o formuladas en lenguaje natural. La búsqueda tradicional, en cambio, depende fuertemente de la coincidencia exacta de términos y requiere mayor esfuerzo por parte del usuario para “adivinar” las palabras correctas.

En conjunto, los experimentos muestran que **la búsqueda semántica complementa y supera a la búsqueda tradicional en escenarios reales**, especialmente cuando el usuario no conoce los términos exactos del documento. No obstante, en consultas muy concretas y bien definidas, la búsqueda por palabras clave sigue siendo eficaz y más controlable.


#### 6) Búsqueda híbrida

La búsqueda híbrida es un enfoque que combina la búsqueda léxica tradicional con la búsqueda semántica basada en vectores para mejorar la calidad de los resultados. Elasticsearch ofrece de forma nativa este tipo de búsqueda híbrida mediante el uso de RRF (Reciprocal Rank Fusion), que permite fusionar rankings procedentes de distintos tipos de consultas (por ejemplo, una consulta textual y una vectorial) en un único ranking final. En este proyecto se intentó inicialmente implementar este mecanismo nativo, pero no fue posible utilizarlo debido a las limitaciones de licencia de Elasticsearch. Por este motivo, se optó por un enfoque alternativo de búsqueda híbrida, basado en la combinación de ambas señales dentro de una única consulta, controlando su influencia relativa mediante pesos (boost).

Para la parte léxica he reutilizado la lógica del buscador clásico que ya tenía implementado. Utilizo una consulta query_string sobre los campos title y content, dando más peso al título mediante un boost (title^2). Esta consulta se incluye dentro de una cláusula should y se le asigna un peso global mediante la propiedad boost (por ejemplo, un 70%). De esta manera, los documentos que coinciden bien a nivel textual incrementan su puntuación, pero no se excluyen aquellos que puedan ser relevantes por otras vías.

La búsqueda semántica se implementa calculando el embedding de la consulta con el mismo modelo usado durante la indexación. Ese vector se utiliza en una cláusula knn sobre el campo title_embedding, que permite recuperar documentos con títulos semánticamente similares, aunque no compartan exactamente las mismas palabras. A esta parte vectorial también se le asigna un peso (por ejemplo, un 30%), lo que me permite controlar su influencia en el ranking final.

Los filtros de fecha se aplican de forma consistente en ambas partes de la búsqueda. Por un lado, se añaden como filtros del bool principal, de modo que no afectan al score léxico. Por otro, se incluyen también dentro de la cláusula knn, para asegurar que la búsqueda vectorial solo considere documentos dentro del rango temporal solicitado. Esto garantiza que tanto la recuperación léxica como la semántica trabajen sobre el mismo subconjunto de documentos.

Además, he mantenido funcionalidades ya presentes en el buscador original, como la paginación, el resaltado de resultados y el sistema de sugerencias. El highlight sigue funcionando sobre los campos textuales para facilitar la interpretación de por qué un documento ha sido recuperado, mientras que el term suggester ayuda a corregir posibles errores tipográficos en la consulta del usuario.

Por último, para que la combinación de ambas señales tenga sentido, la búsqueda híbrida se ejecuta ordenando por _score. De esta forma, el ranking final refleja realmente la combinación ponderada entre la relevancia léxica y la similitud semántica: los documentos con coincidencias exactas siguen apareciendo bien posicionados, pero también pueden emerger resultados relevantes que solo se detectan gracias a la información semántica contenida en los embeddings.

In [ ]:
import re
from typing import Any

def search_news_hybrid(
    user_query: str,
    date_from: str | None = None,
    date_to: str | None = None,
    page: int = 1,
    size: int = 10,
    sort_field: str = "_score",
    sort_dir: str = "desc",
    highlight: bool = True,
    fuzzy: bool = False,
    enable_suggest: bool = True,
    lexical_boost: float = 0.7,
    vector_boost: float = 0.3,
    # KNN params
    k: int = 50,
    num_candidates: int = 200,
) -> dict[str, Any]:

    if not user_query or not user_query.strip():
        raise ValueError("user_query no puede estar vacío")

    page = max(1, int(page))
    size = max(1, int(size))
    from_ = (page - 1) * size

    sort_dir = (sort_dir or "desc").lower()
    if sort_dir not in ("asc", "desc"):
        sort_dir = "desc"

    # Parte léxica (con boost)
    # OJO: query_string no tiene boost interno como match, pero se puede envolver en "boosting"
    # o (más simple) meterlo en should con "boost" usando un wrapper "constant_score".
    # Aquí usamos "query_string" dentro de "should" y controlamos peso con "boost" en el wrapper.
    lexical_clause = {
        "bool": {
            "should": [
                {
                    "query_string": {
                        "query": user_query,
                        "fields": ["title^2", "content"],
                        "default_operator": "AND",
                        "lenient": True,
                        "analyze_wildcard": True
                    }
                }
            ],
            "boost": lexical_boost
        }
    }

    # Filtros (fechas)
    filters: list[dict[str, Any]] = []
    if date_from or date_to:
        r: dict[str, Any] = {}
        if date_from:
            r["gte"] = date_from
        if date_to:
            r["lte"] = date_to
        filters.append({"range": {"date": r}})

    # Query principal (bool)
    # Ponemos la parte léxica en should y dejamos el filtro en filter.
    # minimum_should_match=0 porque el knn ya puede traer resultados aunque el texto no matchee.
    bool_query: dict[str, Any] = {
        "bool": {
            "should": [lexical_clause],
            "filter": filters,
            "minimum_should_match": 0
        }
    }

    # Fuzzy (opcional): añade un should extra que refuerza recall léxico
    if fuzzy:
        bool_query["bool"]["should"].append({
            "multi_match": {
                "query": user_query,
                "fields": ["title^2", "content"],
                "fuzziness": "AUTO",
                "operator": "AND",
                "boost": lexical_boost  # mismo peso que la parte léxica
            }
        })

    # Embedding para kNN (sobre title_embedding)
    query_vector = model.encode([user_query], show_progress_bar=False, normalize_embeddings=True)[0]

    knn_clause = [{
        "field": "title_embedding",
        "query_vector": query_vector.tolist() if hasattr(query_vector, "tolist") else list(query_vector),
        "k": int(k),
        "num_candidates": int(num_candidates),
        "boost": float(vector_boost),
        "filter": {"bool": {"filter": filters}} if filters else None
    }]

    # Limpieza: si no hay filtros, quita filter=None (ES no lo quiere)
    if knn_clause[0].get("filter") is None:
        knn_clause[0].pop("filter", None)

    # Highlight
    highlight_obj = None
    if highlight:
        highlight_obj = {
            "pre_tags": ["<mark>"],
            "post_tags": ["</mark>"],
            "fields": {
                "title": {"number_of_fragments": 0},
                "content": {"fragment_size": 160, "number_of_fragments": 3}
            }
        }

    # Suggest
    suggest_obj = None
    if enable_suggest:
        tokens = re.findall(r"[0-9A-Za-zÁÉÍÓÚÜÑáéíóúüñ]+", user_query)
        suggest_text = " ".join(tokens).strip()
        if suggest_text:
            suggest_obj = {
                "title_suggest": {
                    "text": suggest_text,
                    "term": {"field": "title", "suggest_mode": "popular", "min_word_length": 3}
                },
                "content_suggest": {
                    "text": suggest_text,
                    "term": {"field": "content", "suggest_mode": "popular", "min_word_length": 3}
                }
            }

    # Sort (recomendado: _score)
    if sort_field in ("_score", "score"):
        sort_obj = [{"_score": {"order": sort_dir}}]
    else:
        unmapped_type = "date" if sort_field == "date" else "keyword"
        sort_obj = [{sort_field: {"order": sort_dir, "unmapped_type": unmapped_type}}]

    # Ejecutar búsqueda híbrida (query + knn)
    resp = client.search(
        index=INDEX_NAME,
        query=bool_query,
        knn=knn_clause,
        from_=from_,
        size=size,
        sort=sort_obj,
        highlight=highlight_obj,
        suggest=suggest_obj
    )

    hits = resp.get("hits", {}).get("hits", [])
    total = resp.get("hits", {}).get("total", {}).get("value", 0)

    results = []
    for h in hits:
        src = h.get("_source", {}) or {}
        results.append({
            "score": h.get("_score"),
            "url": src.get("url"),
            "title": src.get("title"),
            "date": src.get("date"),
            "content": src.get("content"),
            "document_type": src.get("document_type"),
            "source_folder": src.get("source_folder"),
            "highlight": h.get("highlight", {})
        })

    suggestions = {}
    if resp.get("suggest"):
        for key, arr in resp["suggest"].items():
            opts = []
            for entry in arr:
                for opt in entry.get("options", []):
                    if "text" in opt:
                        opts.append(opt["text"])
            # uniq
            seen, uniq = set(), []
            for o in opts:
                if o not in seen:
                    seen.add(o)
                    uniq.append(o)
            suggestions[key] = uniq

    return {
        "page": page,
        "size": size,
        "total": total,
        "pages": (total + size - 1) // size,
        "took_ms": resp.get("took"),
        "results": results,
        "suggestions": suggestions,
        "raw_query": {
            "query": bool_query,
            "knn": knn_clause,
            "sort": sort_obj,
            "highlight": highlight_obj,
            "suggest": suggest_obj,
            "from": from_,
            "size": size
        }
    }


Probamos con el caso 2 del apartado anterior, en el que fallaba el buscador léxico

In [51]:
res = search_news_hybrid(
    "CNMC sancion Endesa",
    size=10,
    lexical_boost=0.4,
    vector_boost=0.6,
    sort_field="_score"
)

print("Total:", res["total"])
for r in res["results"][:5]:
    print(r["score"], r["date"], r["title"])

Total: 50
0.4827682 2024-07-10 La CNMC inicia un expediente sancionador contra empresas de Endesa por posibles prácticas anticompetitivas
0.45879415 2025-04-30 Competencia autoriza la opa del BBVA al Sabadell y deja la operación en manos del Gobierno
0.4529676 2025-10-17 Fracasa la OPA hostil del BBVA al Sabadell
0.44816658 2023-05-03 Repsol lanza las primeras estaciones de servicio para repostar diésel 100% renovable
0.4478946 2026-01-02 Apollo, Cheyne y Tikehau pactan con los acreedores sacar a flote Torraspapel y recapitalizarla


# Parte 4 – Vectorización

Para que un algoritmo de machine learning o un sistema de recuperación de información pueda trabajar con texto, primero debemos convertir los documentos en representaciones numéricas. La forma más común de hacerlo es mediante la **matriz de términos y documentos** (también llamada matriz documento-término o DTM), una estructura donde cada fila representa un documento de nuestra colección y cada columna representa un término único del vocabulario. Cada celda contiene un valor numérico que indica la importancia o presencia de ese término en ese documento. La forma más simple de construir esta matriz es mediante el **Bag of Words** (bolsa de palabras), donde cada celda contiene simplemente la frecuencia del término en el documento (TF o Term Frequency), ignorando completamente el orden de las palabras. Por ejemplo, si tenemos 1000 documentos y un vocabulario de 5000 términos únicos (después de filtrar stopwords y términos raros), nuestra matriz será de 1000×5000. Esta matriz suele ser muy dispersa (sparse), es decir, contiene muchos ceros, ya que cada documento solo utiliza una pequeña fracción del vocabulario total, lo cual permite almacenarla eficientemente en formato sparse en lugar de como array denso.


El problema del simple conteo de frecuencias (TF) es que las palabras muy comunes como "después", "entonces" o "muy" tendrán frecuencias altas pero aportan poco significado para distinguir documentos. Para solucionar esto se utiliza **TF-IDF** (Term Frequency - Inverse Document Frequency), un esquema de ponderación que pondera cada término según dos criterios: 

(1) su frecuencia en el documento actual (TF), porque un término que aparece muchas veces en un documento probablemente es relevante para ese documento, y 

(2) su rareza en la colección completa (IDF), penalizando términos que aparecen en muchos documentos porque son menos discriminativos. 

a fórmula básica es `TF-IDF = TF × IDF`, donde `IDF = log(N / df)`, siendo N el número total de documentos y df (document frequency) el número de documentos que contienen el término. Así, un término que aparece 10 veces en un documento (TF alto) pero también aparece en el 90% de los documentos (IDF bajo) tendrá un TF-IDF menor que un término que aparece 5 veces pero solo en el 2% de los documentos. Esta ponderación hace que términos específicos y relevantes tengan valores altos, mientras que términos genéricos o stopwords tengan valores bajos. TF-IDF es fundamental tanto para búsqueda (identificar documentos relevantes) como para análisis (descubrir qué términos caracterizan mejor cada documento o colección). Scikit-learn proporciona `TfidfVectorizer` que combina todo el proceso (tokenización, conteo, cálculo TF-IDF y filtrado) en una sola clase, y también permite hacerlo en dos pasos con `CountVectorizer` (matriz TF) seguido de `TfidfTransformer` (conversión a TF-IDF).

Nos piden filtrar mediante stop-words, por lo que necesitamos descargarnos nltk para obtener una lista con las palabras stopword en español

In [17]:
import nltk

# Descargamos las stopwords de NLTK
nltk.download('stopwords')
stopwords_sp = nltk.corpus.stopwords.words('spanish')

[nltk_data] Downloading package stopwords to /home/jose/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [18]:
# Clase que nos calcula el TF
from sklearn.feature_extraction.text import CountVectorizer

Para poder calcular TF y TF-IDF obtenemos el contenido de todos los documentos en una lista, de manera que cada valor de la lista es un "documento"

In [19]:
from elasticsearch import helpers

def get_all_contents(index_name: str = INDEX_NAME) -> list[str]:
    contents: list[str] = []

    for doc in helpers.scan(
        client,
        index=index_name,
        query={
            "_source": ["content"],
            "query": {"match_all": {}}
        },
        preserve_order=False
    ):
        src = doc.get("_source", {}) or {}
        text = src.get("content") or ""
        contents.append(text)

    return contents

all_contents = get_all_contents()
print("Nº documentos:", len(all_contents))
print("Ejemplo content:", all_contents[0][:200])

Nº documentos: 1282
Ejemplo content: AVA-Asaja asegura que se perderán 75.000 toneladas de cítricos por exceso de humedad mientras La Unió alerta del peligro de aparición de hongos, por lo que pedirá reparto “gratuito y urgente” de fungi


Nos piden filtrar por stopword y eliminar aquellas palabras que aparezcan en menos de 10 documentos. Para lo primero le tenemos que pasar la lista de stopwords por el parámetro `stop_words` a `CountVectorizer`. Para lo segundo usamos el parámetro `min_df` (minimum document frequency) para ignorar términos que aparezcan en menos de un número dado de documentos.

In [20]:
count_vect = CountVectorizer(stop_words=stopwords_sp, min_df=2)
X_counts = count_vect.fit_transform(all_contents) # lista con el content de cada documento

print("Forma de la matriz:", X_counts.shape)

Forma de la matriz: (1282, 12488)


Mostrar los 100 términos más repetidos en la colección

In [21]:
import numpy as np

n = 100

# Suma de frecuencias por columna (es decir, por término)
term_frequencies = np.array(X_counts.sum(axis=0)).flatten()

# Obtener los nombres de los términos
terms = count_vect.get_feature_names_out()

# Crear lista de (término, frecuencia)
term_freq_pairs = list(zip(terms, term_frequencies))

# Ordenar por frecuencia descendente
term_freq_pairs.sort(key=lambda x: x[1], reverse=True)

# Mostrar los n términos más frecuentes
count = 0
for term, freq in term_freq_pairs:
    print(f"{term}: {freq}")
    count += 1
    if count >= n:
        break

euros: 3198
año: 2017
canarias: 1780
millones: 1455
mes: 1168
vivienda: 1102
años: 1088
si: 999
000: 945
según: 925
españa: 891
2024: 876
sector: 770
personas: 757
datos: 674
parte: 668
respecto: 660
así: 637
además: 636
mayor: 636
precios: 634
mientras: 633
menos: 618
2025: 617
meses: 599
gobierno: 595
total: 595
trimestre: 580
10: 579
empleo: 570
media: 556
mismo: 556
cada: 548
nacional: 545
precio: 527
caso: 522
2023: 513
empresas: 513
anterior: 506
incremento: 502
mercado: 496
dos: 481
puede: 481
crecimiento: 480
número: 461
tenerife: 460
viviendas: 455
12: 453
15: 449
pasado: 446
solo: 446
social: 443
trabajo: 443
comunidad: 440
tasa: 433
alquiler: 432
islas: 432
país: 431
cuenta: 426
subida: 423
aunque: 422
madrid: 415
ser: 411
economía: 410
pensión: 404
comunidades: 398
11: 394
turismo: 389
aumento: 387
gran: 386
14: 383
servicios: 382
días: 380
canaria: 377
20: 372
trabajadores: 359
general: 358
tres: 358
forma: 356
laboral: 355
seguridad: 355
día: 351
pensiones: 351
operacione

Ahora calculamos el TF-IDF (matriz donde cada fila es un documento y cada columna un término)

In [22]:
from sklearn.feature_extraction.text import TfidfTransformer

In [23]:
# Calculamos ahora el TFIDF
tfidf_transformer = TfidfTransformer()
X_TFIDF = tfidf_transformer.fit_transform(X_counts)

# Mostramos el número de textos y el número de tokens únicos
print(X_TFIDF.shape)

(1282, 12488)


10 términos más relevantes de toda la colección

In [ ]:
import numpy as np

# Obtener los nombres de los términos
feature_names = count_vect.get_feature_names_out()

# Calcular la media del TF-IDF de cada término en todos los documentos
# X_TFIDF es una matriz sparse (filas=docs, columnas=términos)
mean_tfidf = np.array(X_TFIDF.mean(axis=0)).flatten()  # promedio por columna

# Emparejar términos con sus puntuaciones TF-IDF promedio
term_tfidf_pairs = list(zip(feature_names, mean_tfidf))

# Ordenar de mayor a menor TF-IDF promedio
term_tfidf_pairs.sort(key=lambda x: x[1], reverse=True)

# Número de términos a mostrar
N = 10

# Mostrar los N términos con mayor TF-IDF promedio
print(f"Top {N} términos por TF-IDF promedio:")
for term, score in term_tfidf_pairs[:N]:
    print(f"{term}: {score:.4f}")

Top 10 términos por TF-IDF promedio:
euros: 0.0433
canarias: 0.0312
año: 0.0269
millones: 0.0263
vivienda: 0.0233
mes: 0.0224
000: 0.0179
años: 0.0176
sector: 0.0172
2024: 0.0167
